<a href="https://colab.research.google.com/github/delhian/NLP_course/blob/master/Home%20Tasks/gpt_2_text_clf_unresolved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Возможности и ограничения языковых моделей

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Тут есть важное действие, которое нужно сделать перед тем, как запускать этот ноутбук**

Если вы делаете это задание в Google Colab, первым делом переключите Runtime в GPU. Это задание нормально посчитается и на CPU, но некоторые из будущих кейсов потребуют GPU (либо вам придётся несколько часов или даже дней, чтобы модель обучилась - и мы не преувиличиваем). Ещё мы рекомендуем переключить язык интерфейса на английский, потому что русская локализация ужасна да и вообще не нужна.

О том, как переключить рантайм: https://www.geeksforgeeks.org/how-to-use-google-colab/

In [2]:
!pip install datasets transformers[torch]==2.10.0 tqdm

     |████████████████████████████████| 287 kB 5.4 MB/s 
     |████████████████████████████████| 660 kB 41.6 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 5.6 MB 13.8 MB/s 
     |████████████████████████████████| 1.2 MB 45.5 MB/s 
     |████████████████████████████████| 125 kB 58.2 MB/s 
     |████████████████████████████████| 243 kB 48.5 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 1.3 MB 35.5 MB/s 
     |████████████████████████████████| 160 kB 52.9 MB/s 
     |████████████████████████████████| 271 kB 54.7 MB/s 


In [3]:
from transformers import AutoModel, AutoTokenizer, GPT2LMHeadModel
import datasets

import torch
from torch.nn import functional as F
from tqdm import tqdm
import warnings

In [4]:
warnings.filterwarnings("ignore")

In [5]:
# import transformers
# print(transformers.__version__)  # should be above or equal 4.0.1

В этом модуле мы с вами познакомились с большими языковыми моделями и обсудили их возможности, а также ограничения. В этом задании вам предлагается поэкспериментировать с одной из больших языковых моделей -- GPT-2 -- и самим убедиться, так ли она хороша, как о ней рассказывают.

## Библиотека transformers

Вы уже сталкивались с библиотекой transformers в этом курсе.

Мы [загрузим gpt-2](https://huggingface.co/transformers/model_doc/gpt2.html) с помощью метода from_pretrained. Так как мы будем использовать её для задачи языкового моделирования, нам потребуется GPT2LMHeadModel, прочитать про неё можно [тут](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel). У GPT-2 свой токенизатор, про него можно почитать [тут](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2tokenizer).

GPT-2 -- большая модель, время ее работы на CPU будет слишком большим, так что мы сразу отправим её на GPU.

In [204]:
model_name = 'gpt2-medium'
device = 'cuda'

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [209]:
%%time
# model = GPT2LMHeadModel.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/gpt2-medium')
model = model.to(device)

CPU times: user 10.6 s, sys: 1.68 s, total: 12.3 s
Wall time: 46 s


### Подсчёт числа параметров модели

Чтобы понять, насколько GPT большая модель, подсчитаем число ее параметров

In [211]:
# TASK: compute number of model parameters
# iterate over model weights and count number of parameters in each of them, then sum it up
# note: you may find model.parameters() method useful
# Our implementation is 2 lines

params = sum([params.numel() for params in model.parameters() ])
# YOUR CODE STARTS

# YOUR CODE ENDS
params

354823168

Как видим, это большое число параметров, на несколько порядок больше, чем те модели, которые вы обучали до этого! Использовать такое в продакшене как правило невозможно из-за больших требований по ресурсам

# Часть 1. Генерация отзывов моделью GPT

В преыдущем задании мы генерировали ровно 1 токен, так как нам нужно было предсказать сентимент, а оба слова "positive" и "negative" есть в словаре GPT.

В этом задании мы пойдём дальше и самостоятельно реализуем top-k сэмплирование, чтобы генерировать отзывы на фильмы, подобные тем, что в датасете IMDB.

Мы будем продолжать отзывы, используя в качестве "затравки" для модели начало реальных ревью.

### Top-k & Top-p sampling

Чтобы генерация текста была более разнообразной, мы будем использовать top k сэмплирование, которое мы изучали в этом юните. Его смысл в том, что на каждом шаге мы перед сэмплированием зануляем вероятности всех слов, кроме k самых вероятных.

In [212]:
def topk_sample(scores, k):
    """
    Sample from logits using multinomial distribution
    Before sampling, all logits except k greatest are zeroed out.
    Args:
        scores: scores for every word in the vocabulary. Must be 1-dimensional: (num_words_in_vocab)
        k: int, how many hypotheses to sample from
    Returns:
        1 draw from dictribution, torch.LongTensor of shape (1)
    """
    assert k >= 1, 'k must be >=1'
    assert scores.ndim == 1, 'logits must have 1 dimension' 

    # TASK: implement top-k sampling
    # First, get top k values and theoir indices using torch.topk
    # 2nd, fill logits with some small value (e. g. 1e-6)
    # Next, move values back to their place. Note that you will find ellipsis (...) and None useful
    # Finally, use softmax to obtain probabilities and get 
    # Our implementation is 6 lines
    
    # YOUR CODE STARTS
    values, indexes = scores.topk(k)
    scores = scores.fill_(1e-6)
    scores[indexes] = values
    predicted_ids = F.softmax(scores)
    # YOUR CODE ENDS
    return predicted_ids

Проверка кода:

In [213]:
logits = torch.randn(5)
for _ in range(10):
    res1 = topk_sample(logits, 1)
    res2  = topk_sample(logits, 1)
    assert res1.ndim == 1
    assert res1.equal(res2)
equal = True
for _ in range(10):
    if not topk_sample(logits, 2).equal(topk_sample(logits, 2)):
        equal = False
assert equal
print('OK!')

OK!


Также мы будем использовать top-p sampling, который мы уже проходили на лекциях.

In [214]:
def topp_sample(scores, p):
    """
    Sample from logits using multinomial distribution
    Before sampling, all logits except those largest
    whose cumsum exceeds p are zeroed out
    Args:
        scores: scores for every word in the vocabulary. Must be 1-dimensional: (num_words_in_vocab)
        p: float, cumulative probability of most high-scored tokens
    Returns:
        1 draw from dictribution, torch.LongTensor of shape (1)
    """
    assert  0 < p < 1, 'p must be between 0 and 1'
    assert scores.ndim == 1, 'logits must have 1 dimension' 

    # TASK: implement top-p sampling
    # 1. sort scores with descending order

    values, indexes = scores.sort(descending = True)
    # 2. get cumulative sum of softmaxed logits

    values_cumsum = torch.cumsum(values, dim =0 )

    # 3. get indices when cumulative sum is larger than p. These indices should be zeroed out

    scores[indexes[values_cumsum > p]] = 1e-6

    # 4. get real indices that should be zeroes out from sorted indices. Use tensor slicing
    # 5. fill these indices with some small value (e. g. -1e6)
    # 6. sample, as you did in top-k sampling
    # Our implementation is 7 lines
    
    # YOUR CODE STARTS
    predicted_ids = F.softmax(scores)
    # YOUR CODE ENDS
    return predicted_ids

Проверка кода:

In [215]:
logits = torch.Tensor([-.1, -.2, .9])
for _ in range(10):
    res1 = topp_sample(logits, 0.8)
    res2  = topp_sample(logits, 0.8)
    assert res1.ndim == 1

    assert res1.equal(res2)
logits = torch.Tensor([.5, .5, .5])
equal = True
for _ in range(10):
    if not topp_sample(logits, 0.8).equal(topp_sample(logits, 0.8)):
        equal = False
assert equal
print('OK!')

OK!


Теперь напишите функцию генерации текста:

In [233]:
def generate_text(model, ids, length, k=None, p=None):
    """
    Generate text with language model with ids as prompt
    Args:
        model: huggingface LM model
        ids: input ids, from tokenizer.encode, must be 2-dimensional
        length: int, how many tokens to geenrate
        k: int, how many hypotheses to sample from in top-k sampling
    Returns:
        token ids from generated text, together with token ids of prompts, 2d LongTensor
    """
    assert length >= 1
    if k and p:
        raise RuntimeError('Cannot use topk and topp sampling simultaneously')
    # TASK: write generation loop
    # For every timestamp:
    # - obtain model output
    # - get logits for last symbol
    # - apply topk sampling to get new index
    # - concatenate it to the ids to form new input
    # in the end, the `ids` tensor will have full generation
    # our implementation is 5 lines
    
    # YOUR CODE STARTS

    ids = model.generate(
        ids.reshape(-1, ids.shape[0]), 
        do_sample=True, 
        max_length=ids.shape[0] + length, 
        repetition_penalty=1.2, 
        pad_token_id=tokenizer.eos_token_id,
        top_k = k,
        top_p = p
    )

    # YOUR CODE ENDS
    return ids.reshape(ids.shape[1]).cpu().detach()

Проверка кода:

In [235]:
ids = torch.randint(0, 1000, (4,)).to(device)
generation = generate_text(model, ids, 4, k=2)

assert generation.shape == torch.Size([8])

generation = generate_text(model, ids, 3, p=.5)

assert generation.shape == torch.Size([7])

### Запускаем генерацию

Зададим модели какой-нибудь prompt, например восторженный отзыв о фильме, и посмотрим, как она продолжит его. Так как модель сэмплит на каждом шаге, нам интересно посмотреть на несколько траекторий сразу. Для этого мы и писали батч-режим в генерации.

In [236]:
prompt = 'What a lovely film !'

Подготовим входные данные для GPT

In [242]:
ids = tokenizer.encode(prompt, return_tensors="pt").squeeze().to(device)

Запускаем генерацию! Можно запустить на 10-20 токенов, можно и больше

In [243]:
n_tries = 5

In [244]:
generations = [generate_text(model, ids, length=40, k=10) for _ in range(n_tries)]

In [245]:
for generation in generations:
    print(tokenizer.decode(generation))
    print('\n' + '=' * 40 + '\n')

What a lovely film!
I hope you liked it and have enjoyed my first ever blogpost. If not, I am looking for your feedback on the post as we are in need of more comments! Please feel free to


What a lovely film! It is not only the best, the most beautiful and also one of my favorites!

I was so surprised how well written everything about this movie really is. The characters are all wonderful to watch


What a lovely film!
It has all the right ingredients and is perfect for any movie night. It's also very funny at times too so if you are looking to laugh, it might be something that interests your friends as


What a lovely film! I was in the UK at that time and had to get it for my friend, so here is his copy :D This review may be edited by me if needed.
<|endoftext|>


What a lovely film! It's one of those movies that makes you realise there are many things in life, and the only ones we're all born with.

In this movie they take me on an epic journey across




А теперь запустим генерацию с top-p сэмплированием

In [246]:
generations = [generate_text(model, ids, length=40, p=0.5).cpu().detach() for _ in range(n_tries)]
for generation in generations:
    print(tokenizer.decode(generation))
    print('\n' + '=' * 40 + '\n')

What a lovely film! The best thing about it is that you can watch the movie in multiple ways. You may want to see all three of them, but I recommend watching one at once and then picking up where we left


What a lovely film! It is very much about the life of Jesus Christ. The script was written by an American writer and directed in English with great care, it's beautiful,the actors are outstanding. I would recommend this


What a lovely film!
"You have no idea what I feel for you, my dear." - The Princess Bride "I'm not so sure that's the right word to use. She doesn't seem like an ordinary


What a lovely film! The characters are wonderful and the story is interesting. It's like watching an old friend from childhood, with whom you've had many adventures together for years now. And if I could give it five stars


What a lovely film!
I have seen this movie many times in my life. I remember when it first came out and thought that the story was great but not enough of an explanation as t

# Часть 2. Few-shot learning

Одной из особенностей GPT, про которую авторы написали, является способность к few-shot learning. В этой части домашнего задания мы на примере хорошо знакомой нам задачи классификации текстов исследуем, как хорошо модель умеет различать сентимент отзывов без дополнительного обучения.

## Датасет.
Мы будем снова использовать датасет imdb, на котором мы уже обучили несколько моделей. Так как обучать саму модель мы не будем (в few-shot парадигме веса не меняются), то можем сразу взять валидационную часть датасета.

In [247]:
text_dataset = datasets.load_dataset("imdb")
texts = text_dataset["test"]["text"]
labels = text_dataset["test"]["label"]

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


  0%|          | 0/3 [00:00<?, ?it/s]

### Метки для задачи классификации на естественном языке
Поскольку языковая модель знает только токены языка и не знает маппинга между индексами и лейблами, нам нужно будет сравнивать лейблы напрямую. С одной стороны, это создаёт дополнительные сложности: модель может предсказать "Positive Sentiment" вместо "positive", поэтому нам потребуется минимальный постпроцессинг, чтобы учесть большинство таких случаев.

In [248]:
mapping = {idx: label for idx, label in enumerate(text_dataset["train"].features["label"].names)}
mapping

{0: 'neg', 1: 'pos'}

### Игрушечный датасет
Чтобы быстрее прототипировать наши вводы (prompts), полезно взять маленькую часть датасета и проверять гипотезы на ней. Так как imdb упорядочен по лейблам, а самих лейблов всего 2, мы берём одинаковое количество примеров с начала и с конца.

In [413]:
def get_toy_dataset(dataset, split, length = 100):
    texts = dataset[split]["text"]
    labels = dataset[split]["label"]
    small_texts = texts[:length // 2] + texts[-length//2:]
    small_labels = labels[:length // 2] + labels[-length//2:]
    return small_texts, small_labels

In [414]:
assert len(get_toy_dataset(text_dataset, "test", 100)[0]) == 100

## Few-shot learning

В данном задании мы будем не обучать модель на данных, а использовать уже обученную на огромном корпусе языковую модель, чтобы "угадывать" сентимент отзывов.

Модель будет видеть в качестве затравки (prompt) описание задачи на естественном языке.

Нам потребуется:
- описание задачи
- лейблы
- несколько (хотя бы 3) примеров
- вспомогательные маркеры (разделитель примеров и маркер конца входа)

Например:

```
Translate from English to French
sea otter => loutre de mer
peppermint => menthe poivrée
plush girafe  => 
```

Первая строка здесь -- описание задачи. Мы решаем задачу классификации, поэтому нам нужно
описать задачу примерно как `To which category does the text belong? positive , negative `

Вторая и третья строки -- примеры входа и выхода. В нашем случае примерами входа и выхода будут служить тексты ревью и метки классов positive и negative (а не 0 и 1, как раньше).

Наконец, последняя строка -- это тот текст, который модель должна классифицировать. Модель уже "видела" выше пример того, что после маркера конца входа идёт сентимент, и мы ожидаем, что она сможет выучить эту закономерность.

В данном задании часть входа будет написана за вас, вам надо будет придумать обучающие примеры для few-shot learning'а

In [251]:
marker = ' => '
separator = '\n'

In [252]:
labels = ["positive" , "negative"]
labels_text = " , ".join(labels)
task_description = f'To which category does the text belong?: {labels_text} ' + separator

In [253]:
lbl = text_dataset['train']['label']
texts =  text_dataset['train']['text']

In [120]:
import random

In [545]:
import random

noex = 5
random_negative = random.sample(texts[12500: 24999], noex)
random_positive = random.sample(texts[0: 12499], noex)

In [546]:
examples = [
    # TASK define examples
    # They should be a tuple with text and label
    # label should be from "labels"
    # Get 3-10 different examples, they should cover both positive 
    # and negative reviews
    # YOUR CODE STARTS
(' '.join(x.split(' ')[:30]), "negative") for x in random_negative] + [(' '.join(x.split(' ')[:30]), "positive") for x in random_positive
    # YOUR CODE ENDS
]

In [547]:
assert len(examples) > 1, 'Write at least two different examples'
for ex in examples:
    assert ex[1] in labels

In [548]:
input = task_description  + separator.join([marker.join(example) for example in examples])

In [549]:
print(input)

To which category does the text belong?: positive , negative 
R O B O T J O X.<br /><br />Burn the master.<br /><br />Grotesquely horrible.<br /><br />No ending; no closure.<br /><br />Completely and utterly the worst movie ever made.<br /><br => negative
This is by far the worst movie I have ever seen in the cinema!! Could not wait for it to end. To make matters worse it is given a 12A => negative
I believe that war films should try to convey the terror of war, avoid idealism and respect some rudimentary military principles. Zvezda barely does the first. Zvezda being a Russian => negative
This had high intellectual pretensions.The main lead intends to give a "deep" "meaningful" rendering(with voice over for his frames of mind naturally) and he was certainly influenced by the fifties/sixties => negative
"The Cobweb" is an example of many examples of movies that feature strong, sometimes noteworthy performances and high points, but unfortunately are shattered and slowed down drastically

Сравните получившийся инпут с примером выше. Мы получили то, что надо!

In [550]:
n_chars = 50

In [551]:
def get_prompt(beginning, separator, text, marker) -> str:
    return ' '.join([input,  separator, text[:n_chars] + text[-n_chars:], marker])

## Время проверить работу нашей модели на маленьком датасете!

In [552]:
small_texts, small_labels = get_toy_dataset(text_dataset, "test")

Так как мы предсказываем сентимент текста, каждый из который задаётся одним токеном, то нам
- не надо сэмплировать. Мы хотим, чтобы модель честно учитывала вероятности для токенов
- достаточно сделать один шаг генерации. Модель должна будет предсказать лейбл текста, а он, как мы выяснили, однотокенный.

Таким образом, нам просто надо сделать один шаг жадного предсказания (greedy decoding).

In [573]:
def predict_on_text(model, ids) -> int:
    """
    Run Language Model on text and use logits from last time 
    step to predict sentence category with greedy decoding
    Args:
        model: huggingface LM model
        ids: LongTensor . Text encoded with tokenizer.
    Returns:
        model prediction, index of most probable word
    """
    # TASK: run a model and get index most probable logit
    # HINT: do not forget to detach tensors
    # HINT 2: GPT-2 model returns tuple where logits are stored in the
    # 1st item. You do not need 2nd item at all
    # Our implementation is 2 lines
    # YOUR CODE STARTS

    idx = model.generate(
        ids.reshape(-1, ids.shape[0]),
        max_length=ids.shape[0] + 1,
        pad_token_id=tokenizer.eos_token_id,
    )[0][-1]

    # YOUR CODE ENDS
    return idx.cpu().detach().item()

Проверка кода:

In [574]:
ids = tokenizer.encode('hello world to ', return_tensors="pt",).squeeze()
# print(ids.shape)
prediction = predict_on_text(model, ids.to(device))
assert isinstance(prediction, int)

In [575]:
# prediction = predict_on_text(model, ids.to(device))

# for p in prediction:
#     print(tokenizer.decode(p))
#     print('\n' + '=' * 40 + '\n')

In [576]:
def predict_on_dataset(model, texts, target, mapping):
    """
    Run Language Model on a dataset and use its predictions as
    labels for sentences
    Args:
        model: huggingface LM model
        texts: List[str], texts of dataset
        target: List[int], indices of classes
        mapping: Dict[int, str] mapping from class indices to class labels
    Returns:
        model predictions, "as is", List[str]
        target labels, after mapping, List[str]
    """
    if len(texts) != len(target):
        raise RuntimeError('Texts and target lengths mismatch')
    # max_length has additional -1 because we will generate exactly 1 token 
    # with LM
    max_length = model.config.n_ctx - 1
    predictions = []
    labels = []
    for idx in tqdm(range(len(texts))):
        text, label = texts[idx], target[idx]
        ids = tokenizer.encode(
            get_prompt(input, separator, text, marker), 
            add_special_tokens=False, 
            return_tensors="pt",
            max_length=max_length
        )
        idx = predict_on_text(model, ids.to(device))
        predictions.append(tokenizer.decode([idx]))  # generation.cpu().numpy()[0][-1]
        labels.append(mapping[label]) 
    return predictions, labels

In [577]:
predictions, target = predict_on_dataset(model, small_texts[:10], small_labels[:10], mapping)

100%|██████████| 10/10 [00:02<00:00,  3.85it/s]


### Замер качества

Посмотрим, что выдаёт нам языковая модель:

In [572]:
import numpy as np
print(np.unique(predictions))

['To']


Можно заметить, что предсказанные метки не совсем похожи на pos и neg, которые были у нас в таргете. Напишем простую функцию, которая нормализует предсказания модели

In [270]:
def normalize_prediction(raw_output: str) -> str:
    """
    Helper that transforms model prediction to normal
    For example, ' positive' -> 'pos', ' Negative' -> neg
    Args:
        raw_output: str, output from language model
    Returns:
        normalized value: no leading/trailing spaces, lowercase,
        at most 3 chars long
    """
    # TASK: write prediction normalizer
    # You need to remove spaces, lowercase and get only 3 leading chars
    # Our implementation is 1 line
    # YOUR CODE STARTS

    normalized_output = [x.strip().lower()[:3] for x in raw_output]

    # YOUR CODE ENDS
    return normalized_output

Напишем функцию, вычисляющую точность:

In [ ]:
def accuracy(predictions, labels) -> float:
    """
    Helper that transforms model prediction to normal
    For example, ' positive' -> 'pos', ' Negative' -> neg
    Args:
        predictions: List[str], model predictions, should be labels in natural language
        labels: List[str], target labels
    Returns:
        accuracy, float from [0, 1]
    """
    if not labels:
        # sanity check to avoid zero division
        return 0
    if len(predictions) != len(labels):
        raise ValueError(f'Predictions and labels have mismatched length')
    total = len(predictions)
    match = 0
    # TASK: calculate accuracy
    # For every pair of predicted and real labels, check that they coincide.
    # You can use zip() method for iteration over two sequences
    # simultaneously.
    # Our implementation is 3 lines
    # YOUR CODE STARTS

    # YOUR CODE ENDS
    return accuracy

In [ ]:
normalized_predictions = [normalize_prediction(label) for label in predictions]

In [ ]:
accuracy(normalized_predictions, target)

### Ура, модель без обучения работает лучше случайного угадывания!

Тем не менее, это заметно хуже, чем у полносвязной нейросети. **Учитывая требования по ресурсам и время 
работы, использование такой модели в реальных задачах непрактично!**

## А теперь замер на всем тестовом сете:

In [ ]:
predictions, target = predict_on_dataset(model, texts, labels, mapping)
normalized_predictions = [normalize_prediction(label) for label in predictions]
accuracy(normalized_predictions, target)

Все задание считается выполненным, если вы достигли accuracy 0.6 и выше на тестовом датасете. 

Данный пример призван продемонстрировать, что большая языковая модель хоть и справляется с задачей без лишних данных, но делает это заметно хуже специально созданных для задачи моделей, к тому же работает непростительно долго.

### Резюме

В этом задании мы применили большие языковые модели на практике и решили с её помощью 2 задачи:
- conditional генерация текста
- few-shot классификация текстов

Мы увидели, что модель генерирует связный текст и может решать задачу классификации без изменений своих весов.

Несмотря на возможности языковых моделей, их применение на практике всё ещё осложнено большими потребляемыми ресурсами, а также невозможностью интерпретировать предсказания.